In [31]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [11]:
data = pd.read_csv('D:\\Coding_data\\yellow_tripdata_2019-1_V2.csv')

# data = data[data['PULocationID'] != 1]
# data = data[data['DOLocationID'] != 1]
# data = data[data['PULocationID'] < 264]
# data = data[data['DOLocationID'] < 264]
# data = data[data['RatecodeID'] == 1]
# data = data[data['fare_amount'] > 0]
# data['trip_distance'] = (data['trip_distance'] * 1.609344).round(decimals=2) # getting KM
# data = data[data['trip_distance'] >= 0.5]
# data = data[data['trip_distance'] < 100]
# data = data[data['fare_amount'] < 150]
# data = (data[data['fare_amount'] >= 2.5])
# data = data[data['payment_type'] <= 2]

irrelevant_features = ['RatecodeID', 'payment_type']
data.drop(irrelevant_features, inplace=True, axis=1)
pd.set_option('use_inf_as_na', True)
data = data.replace([np.inf, -np.inf], 0).dropna(subset=data.columns, how="all")
data.dropna()

   tpep_pickup_datetime  tpep_dropoff_datetime  trip_distance  PULocationID  \
0            1546303600             1546304000           2.41           151   
1            1546304387             1546305539           4.18           239   
2            1546302088             1546302517           2.09           163   
3            1546302721             1546303539           5.95           229   
4            1546304252             1546304972           3.38           141   

   DOLocationID  fare_amount  trip_time  distance/time  
0           239          7.0        400       0.006025  
1           246         14.0       1152       0.003628  
2           229          6.5        429       0.004872  
3             7         13.5        818       0.007274  
4           234         10.0        720       0.004694  


In [52]:
data.shape

(7094959, 8)

In [47]:
data.isna().sum()

tpep_pickup_datetime     0
tpep_dropoff_datetime    0
trip_distance            0
PULocationID             0
DOLocationID             0
fare_amount              0
trip_time                0
distance/time            0
dtype: int64

In [48]:
data.notnull().values.all()

True

In [49]:
np.isfinite(data).sum()

tpep_pickup_datetime     7094959
tpep_dropoff_datetime    7094959
trip_distance            7094959
PULocationID             7094959
DOLocationID             7094959
fare_amount              7094959
trip_time                7094959
distance/time            7094959
dtype: int64

In [50]:
np.isnan(data).all()

tpep_pickup_datetime     False
tpep_dropoff_datetime    False
trip_distance            False
PULocationID             False
DOLocationID             False
fare_amount              False
trip_time                False
distance/time            False
dtype: bool

In [51]:
data.isnull().values.all()

False

In [ ]:
print(data.head())

In [12]:
predict = 'fare_amount'

X = data.drop(predict, axis=1)
y = data[predict]

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)
X_mean = scaler.mean_
X_std = scaler.scale_

In [29]:
X_valid.shape

(1330305, 7)

In [13]:
def save_to_multiple_csv_files(data, name_prefix, header=None, n_parts=10):
    ny_dir = os.path.join("datasets", "ny_taxi")
    os.makedirs(ny_dir, exist_ok=True)
    path_format = os.path.join(ny_dir, "my_{}_{:02d}.csv")

    filepaths = []
    m = len(data)
    for file_idx, row_indices in enumerate(np.array_split(np.arange(m), n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)
        filepaths.append(part_csv)
        with open(part_csv, "wt", encoding="utf-8") as f:
            if header is not None:
                f.write(header)
                f.write("\n")
            for row_idx in row_indices:
                f.write(",".join([repr(col) for col in data[row_idx]]))
                f.write("\n")
    return filepaths

In [17]:
train_data = np.c_[X_train, y_train]
valid_data = np.c_[X_valid, y_valid]
test_data = np.c_[X_test, y_test]
header_cols = data.columns
header = ",".join(header_cols)

train_filepaths = save_to_multiple_csv_files(train_data, "train", header, n_parts=20)
valid_filepaths = save_to_multiple_csv_files(valid_data, "valid", header, n_parts=10)
test_filepaths = save_to_multiple_csv_files(test_data, "test", header, n_parts=10)

In [18]:
pd.read_csv(train_filepaths[0]).head()

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,fare_amount,trip_time,distance/time
0,1.548430e+09,1.548431e+09,2.72,163.0,48.0,981.0,0.002773,11.0
1,1.547678e+09,1.547678e+09,4.99,234.0,48.0,748.0,0.006671,11.5
2,1.548235e+09,1.548235e+09,0.64,161.0,230.0,284.0,0.002254,5.0
3,1.546638e+09,1.546638e+09,2.43,163.0,239.0,405.0,0.006000,7.0
4,1.548715e+09,1.548716e+09,2.06,186.0,161.0,668.0,0.003084,8.5


In [19]:
with open(train_filepaths[0]) as f:
    for i in range(5):
        print(f.readline(), end="")

tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,fare_amount,trip_time,distance/time
1548430045.0,1548431026.0,2.72,163.0,48.0,981.0,0.0027726809378185533,11.0
1547677514.0,1547678262.0,4.99,234.0,48.0,748.0,0.006671122994652407,11.5
1548234909.0,1548235193.0,0.64,161.0,230.0,284.0,0.0022535211267605635,5.0
1546637764.0,1546638169.0,2.43,163.0,239.0,405.0,0.006,7.0


In [20]:
train_filepaths

['datasets\\ny_taxi\\my_train_00.csv',
 'datasets\\ny_taxi\\my_train_01.csv',
 'datasets\\ny_taxi\\my_train_02.csv',
 'datasets\\ny_taxi\\my_train_03.csv',
 'datasets\\ny_taxi\\my_train_04.csv',
 'datasets\\ny_taxi\\my_train_05.csv',
 'datasets\\ny_taxi\\my_train_06.csv',
 'datasets\\ny_taxi\\my_train_07.csv',
 'datasets\\ny_taxi\\my_train_08.csv',
 'datasets\\ny_taxi\\my_train_09.csv',
 'datasets\\ny_taxi\\my_train_10.csv',
 'datasets\\ny_taxi\\my_train_11.csv',
 'datasets\\ny_taxi\\my_train_12.csv',
 'datasets\\ny_taxi\\my_train_13.csv',
 'datasets\\ny_taxi\\my_train_14.csv',
 'datasets\\ny_taxi\\my_train_15.csv',
 'datasets\\ny_taxi\\my_train_16.csv',
 'datasets\\ny_taxi\\my_train_17.csv',
 'datasets\\ny_taxi\\my_train_18.csv',
 'datasets\\ny_taxi\\my_train_19.csv']

In [21]:
valid_filepaths

['datasets\\ny_taxi\\my_valid_00.csv',
 'datasets\\ny_taxi\\my_valid_01.csv',
 'datasets\\ny_taxi\\my_valid_02.csv',
 'datasets\\ny_taxi\\my_valid_03.csv',
 'datasets\\ny_taxi\\my_valid_04.csv',
 'datasets\\ny_taxi\\my_valid_05.csv',
 'datasets\\ny_taxi\\my_valid_06.csv',
 'datasets\\ny_taxi\\my_valid_07.csv',
 'datasets\\ny_taxi\\my_valid_08.csv',
 'datasets\\ny_taxi\\my_valid_09.csv']

In [22]:
test_filepaths

['datasets\\ny_taxi\\my_test_00.csv',
 'datasets\\ny_taxi\\my_test_01.csv',
 'datasets\\ny_taxi\\my_test_02.csv',
 'datasets\\ny_taxi\\my_test_03.csv',
 'datasets\\ny_taxi\\my_test_04.csv',
 'datasets\\ny_taxi\\my_test_05.csv',
 'datasets\\ny_taxi\\my_test_06.csv',
 'datasets\\ny_taxi\\my_test_07.csv',
 'datasets\\ny_taxi\\my_test_08.csv',
 'datasets\\ny_taxi\\my_test_09.csv']

--- Building an Input Pipeline

In [33]:
filepath_dataset = tf.data.Dataset.list_files(train_filepaths, seed=42)

In [34]:
for filepath in filepath_dataset:
    print(filepath)

tf.Tensor(b'datasets\\ny_taxi\\my_train_05.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\ny_taxi\\my_train_16.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\ny_taxi\\my_train_01.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\ny_taxi\\my_train_17.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\ny_taxi\\my_train_00.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\ny_taxi\\my_train_14.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\ny_taxi\\my_train_10.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\ny_taxi\\my_train_02.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\ny_taxi\\my_train_12.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\ny_taxi\\my_train_19.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\ny_taxi\\my_train_07.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\ny_taxi\\my_train_09.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\ny_taxi\\my_train_13.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\ny_taxi\\my_train_15.csv', sh

In [35]:
n_readers = 5
dataset = filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
    cycle_length=n_readers)

In [36]:
for line in dataset.take(5):
    print(line.numpy())

b'1546324359.0,1546325409.0,9.38,100.0,87.0,1050.0,0.008933333333333335,19.5'
b'1547932233.0,1547933370.0,5.95,48.0,263.0,1137.0,0.005233069481090589,14.5'
b'1548538122.0,1548541125.0,16.88,164.0,62.0,3003.0,0.005621045621045621,38.0'
b'1547315560.0,1547315802.0,0.8,79.0,113.0,242.0,0.0033057851239669425,4.5'
b'1548199433.0,1548200404.0,9.3,7.0,238.0,971.0,0.009577754891864059,18.0'


In [37]:
n_inputs = 7 # X_train.shape[-1]

@tf.function
def preprocess(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(fields[:-1])
    y = tf.stack(fields[-1:])
    return (x - X_mean) / X_std, y

In [38]:
preprocess(b'1546324359.0,1546325409.0,9.38,100.0,87.0,1050.0,0.008933333333333335,19.5')

(<tf.Tensor: shape=(7,), dtype=float32, numpy=
 array([-1.2020956 , -1.2019857 ,  1.2152402 , -0.97327405, -1.0800096 ,
         0.01649805,  0.10888793], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([19.5], dtype=float32)>)

Lets put all preproecessing stuff into one function:

In [39]:
def csv_reader_dataset(filepaths, repeat=1, n_readers=5,
                       n_read_threads=None, shuffle_buffer_size=10000,
                       n_parse_threads=5, batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths).repeat(repeat)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1), cycle_length=n_readers, num_parallel_calls=n_read_threads)
    dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset.prefetch(1)

In [41]:
train_set = csv_reader_dataset(train_filepaths, batch_size=3)
for X_batch, y_batch in train_set.take(2):
    print("X =", X_batch)
    print("y =", y_batch)
    print()

Cause: could not parse the source code:

        lambda filepath: tf.data.TextLineDataset(filepath).skip(1), cycle_length=n_readers, num_parallel_calls=n_read_threads)

This error may be avoided by creating the lambda in a standalone statement.

Cause: could not parse the source code:

        lambda filepath: tf.data.TextLineDataset(filepath).skip(1), cycle_length=n_readers, num_parallel_calls=n_read_threads)

This error may be avoided by creating the lambda in a standalone statement.

X = tf.Tensor(
[[ 1.0882282e+00  1.0889081e+00  8.5837013e-01  1.0490044e+00
  -1.0656188e+00  1.2772492e-01 -1.7295988e-02]
 [ 7.1220666e-01  7.1243417e-01  3.9152142e-01  1.1098248e+00
  -1.0368372e+00  2.3888538e-02 -1.6484628e-03]
 [-4.6415874e-01 -4.6393421e-01  3.0579460e+00 -4.8671079e-01
  -1.1519635e+00  2.2964725e-02  3.2664129e-01]], shape=(3, 7), dtype=float32)
y = tf.Tensor(
[[22.5]
 [14.5]
 [31. ]], shape=(3, 1), dtype=float32)

X = tf.Tensor(
[[-0.5771232  -0.57667327 -0.20326182  1.06420

In [42]:
train_set = csv_reader_dataset(train_filepaths, repeat=None)
valid_set = csv_reader_dataset(valid_filepaths)
test_set = csv_reader_dataset(test_filepaths)

Cause: could not parse the source code:

        lambda filepath: tf.data.TextLineDataset(filepath).skip(1), cycle_length=n_readers, num_parallel_calls=n_read_threads)

This error may be avoided by creating the lambda in a standalone statement.

Cause: could not parse the source code:

        lambda filepath: tf.data.TextLineDataset(filepath).skip(1), cycle_length=n_readers, num_parallel_calls=n_read_threads)

This error may be avoided by creating the lambda in a standalone statement.

Cause: could not parse the source code:

        lambda filepath: tf.data.TextLineDataset(filepath).skip(1), cycle_length=n_readers, num_parallel_calls=n_read_threads)

This error may be avoided by creating the lambda in a standalone statement.

Cause: could not parse the source code:

        lambda filepath: tf.data.TextLineDataset(filepath).skip(1), cycle_length=n_readers, num_parallel_calls=n_read_threads)

This error may be avoided by creating the lambda in a standalone statement.

Cause: could not

In [43]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1),
])

In [44]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))

In [45]:
batch_size = 32
model.fit(train_set, steps_per_epoch=len(X_train) // batch_size, epochs=10,
          validation_data=valid_set)

Train for 124716 steps
Epoch 1/10
 37296/124716 [=======>......................] - ETA: 4:42 - loss: nan

KeyboardInterrupt: 

In [ ]:
model.evaluate(test_set, steps=len(X_test) // batch_size)